In [1]:
import pyomo as pyo
from pyomo.environ import *
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from shift_and_coverage import create_shifts, is_shift_covering_period

In [8]:
employee_count = 7
employee_ids = ['emp{}'.format(i) for i in range(employee_count)]
employee_ids

['emp0', 'emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6']

### Set of days in the planning horizon & Time periods
**Intervals_in_horizon is the Set of time periods 1....48 intervals in 30 min**
**days in the horizon are the Set of days in the planning horizon**

In [9]:
weeks_in_horizon = 1 #weeks
horizon_lenght = 2 * weeks_in_horizon # days in horizon
period_length = 60 # interval definition
day_length = 24 * 60 // period_length    # intervals
intervals_in_horizon = list(range(day_length)) # intervals in horizon

days_in_horizon = list(range(horizon_lenght))
day_length

24

### Create demand for each time period

In [4]:
demand = {
    0: 4,
    1: 3,
    2: 3,
    3: 2 ,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 1,
    10: 2,
    11: 2,
    12: 2,
    13: 2,
    14: 1,
    15: 1,
    16: 2,
    17: 2,
    18: 2,
    19: 2,
    20: 2,
    21: 2,
    22: 3,
    23: 4,
    }

In [ ]:
""" demand = {
    0: {hour: demand_value for hour, demand_value in enumerate([4, 3, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4], start=1)},
    1: {hour: demand_value for hour, demand_value in enumerate([4, 3, 3, 2, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 3, 5], start=1)}
}
print("Demand structure:", demand)
 """

### Shift creation, create possible shifts with min and max duration

In [ ]:
shifts =create_shifts(60)
shifts

### Pyomo Model

In [11]:
# Define the Pyomo model
model = ConcreteModel()

# Sets
model.E = Set(initialize=employee_ids)  # Set of employee IDs
model.T = Set(initialize=intervals_in_horizon)  # Set of time periods
#model.S = Set(initialize=create_shifts(period_length))  # Set of possible shifts
# Validation: 
Shifts = [(6,10), (14,8), (21,10)]
model.S = Set(initialize=Shifts)
#model.D = Set(initialize=days_in_horizon)

# Variables
model.x = Var(model.E, model.S, domain=Binary)        # Employees assigned to work shift s 


In [6]:
# Print
print("Employees (E):", model.E.pprint())
print("Time Periods (T):", model.T.pprint())
print("Shifts (S):", model.S.pprint())

E : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    8 : {'emp0', 'emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6', 'emp7'}
Employees (E): None
T : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
Time Periods (T): None
S : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :    3 : {(6, 10), (14, 8), (21, 10)}
Shifts (S): None


In [12]:

# Constraints
def demand_constraint_rule(model, time):
    if time in model.T: 
        return sum(model.x[employee, shift] for employee in model.E for shift in model.S if is_shift_covering_period(shift, time, day_length)) >= demand[time]
    else:
            # If demand for this period is not defined, no constraint is needed
        return Constraint.Skip
    
model.demand_constraint = Constraint(model.T, rule=demand_constraint_rule)

def exclusive_shifts_constraint(model, employee, time):
    return sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, time, day_length)) <= 1

model.exclusive_shifts_constraint = Constraint(model.E, model.T, rule=exclusive_shifts_constraint)

model.obj = Objective(expr=1, sense=minimize)

# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(model)

# Print results
""" if str(results.solver.termination_condition) == "optimal":
    print("Solution found:")
    for employee in model.E:
        for shift in model.S:
            if value(model.x[employee, shift]):
                print(f"Assign {employee} to shift starting at {shift[0]} with duration {shift[1]}")
else:
    print("No feasible solution found.") """


if str(results.solver.termination_condition) == "optimal":
    print("Solution found:")
    
    # Initialize a dictionary to hold employees for each shift
    shift_assignments = {}
    for shift in model.S:
        shift_assignments[shift] = []
        
    # Populate the dictionary with employees assigned to each shift
    for employee in model.E:
        for shift in model.S:
            # Check if the employee is assigned to the shift
            if value(model.x[employee, shift]) > 0.5:  # Assuming a binary variable, adjusted for numerical stability
                shift_assignments[shift].append(employee)
    
    # Sort and print the assignments
    for shift, employees in shift_assignments.items():
        if employees:  # Only print shifts that have employees assigned
            sorted_employees = sorted(employees)  # Sort employees for the current shift
            print(f"Shift starting at {shift[0]} with duration {shift[1]}: {', '.join(sorted_employees)}")
else:
    print("No feasible solution found.")


model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
No feasible solution found.


In [ ]:
print(shift_assignments)

In [ ]:
if str(results.solver.termination_condition) == "optimal":
    print("Solution found:")
    
    # Initialize a dictionary to hold employees for each shift
    shift_assignments = {}
    for shift in model.S:
        shift_assignments[shift] = []
        
    # Populate the dictionary with employees assigned to each shift
    for employee in model.E:
        for shift in model.S:
            # Check if the employee is assigned to the shift
            if value(model.x[employee, shift]) > 0.5:  # Assuming a binary variable, adjusted for numerical stability
                shift_assignments[shift].append(employee)
    
    # Sort and print the assignments
    for shift, employees in shift_assignments.items():
        if employees:  # Only print shifts that have employees assigned
            sorted_employees = sorted(employees)  # Sort employees for the current shift
            print(f"Shift starting at {shift[0]} with duration {shift[1]}: {', '.join(sorted_employees)}")
else:
    print("No feasible solution found.")

In [ ]:
model.x.display()

In [ ]:
""" def exclusive_shifts_constraint(model, employee, time):
    # Employee 'e' cannot work two shifts simultaneously
    return sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, time, day_length)) <= 1

model.exclusive_shifts_constraint = Constraint(model.E, model.T, rule=exclusive_shifts_constraint) """
